In [1]:
import pandas as pd
import sqlite3

In [2]:
with sqlite3.connect('../data/main.db') as conn:
    df = pd.read_sql('select msm_id, resolver, prb_id, src_addr, dst_addr, is_tls, return_code, err, err_msg from results', con=conn)

In [3]:
df.head()

,msm_id,resolver,prb_id,src_addr,dst_addr,is_tls,return_code,err,err_msg
0,22202814,None,10006,192.168.0.106,8.8.8.8,0,NOERROR,0,None
1,22202814,None,10007,192.168.7.24,75.75.75.75,0,NOERROR,0,None
2,22202814,None,10007,192.168.7.24,75.75.76.76,0,NOERROR,0,None
3,22202814,None,10019,192.168.31.41,192.168.31.1,0,NOERROR,0,None
4,22202814,None,10023,172.31.253.253,172.31.0.42,0,NOERROR,0,None


In [4]:
len(df[df['err'] == 0]['prb_id'].unique())

5078

In [5]:
neustar = df[df['resolver'].isin(['156.154.70.1', '156.154.71.1'])]

In [6]:
neustar_err = neustar[neustar['err'] == 1]

In [7]:
neustar_err['err_msg'].value_counts()/len(neustar_err)

{u'timeout': 5000}                                       0.999983
{u'socket': u'connect failed Network is unreachable'}    0.000017
Name: err_msg, dtype: float64

In [8]:
dot = df[df['is_tls'] == 1]

In [9]:
dot_err = dot[dot['err'] == 1]

In [10]:
dot_err

,msm_id,resolver,prb_id,src_addr,dst_addr,is_tls,return_code,err,err_msg
16522983,22217033,None,10007,None,None,1,None,1,{u'timeout': 5000}
16522984,22217033,None,10007,None,None,1,None,1,{u'timeout': 5000}
16522985,22217033,None,10007,None,None,1,None,1,{u'timeout': 5000}
16522986,22217033,None,10019,None,None,1,None,1,{u'TU_BAD_ADDR': True}
16522987,22217033,None,10023,None,None,1,None,1,{u'TU_BAD_ADDR': True}
...,...,...,...,...,...,...,...,...,...
160701866,22299012,91.239.100.100,35730,None,None,1,None,1,{u'TUCONNECT': u'Connection reset by peer'}
160701868,22299012,91.239.100.100,35734,None,None,1,None,1,{u'timeout': 5000}
160701869,22299012,91.239.100.100,35735,None,None,1,None,1,{u'timeout': 5000}
160701870,22299012,91.239.100.100,35742,None,None,1,None,1,{u'TUCONNECT': u'Connection reset by peer'}


In [11]:
dot_err.groupby(['prb_id']).size().sort_values(ascending=False)

prb_id
26718    11180
21569    11180
21290    11180
12159    11180
34327    11180
         ...  
34483        2
15505        1
34096        1
25322        1
34458        1
Length: 4927, dtype: int64

### TLS errors

In [12]:
dot_tuconnect = dot_err[dot_err['err_msg'].str.contains('TUCONNECT')]

In [13]:
dot_tuconnect.groupby(['prb_id']).size().sort_values(ascending=False)

prb_id
26451    11180
22955     9784
31088     9784
33408     9784
20461     9778
         ...  
16737        1
31116        1
23272        1
24422        1
18393        1
Length: 4851, dtype: int64

In [14]:
# Filter Uncensored DNS
with sqlite3.connect('../data/main.db') as conn:
    resolvers = pd.read_sql('select * from resolvers', con=conn)

In [15]:
resolvers

,id,name,ipv4,ipv4_2,tls
0,1,CleanBrowsing,185.228.168.168,185.228.169.168,True
1,2,Cloudflare 1.1.1.1,1.1.1.1,1.0.0.1,True
2,3,Comodo Secure DNS,8.26.56.26,8.20.247.20,False
3,4,CZ.NIC ODVR,217.31.204.130,193.29.206.206,False
4,5,Oracle + Dyn,216.146.35.35,216.146.36.36,False
5,6,DNS.WATCH,84.200.69.80,84.200.70.40,False
6,7,Google Public DNS,8.8.8.8,8.8.4.4,True
7,8,Neustar UltraRecursive,156.154.70.1,156.154.71.1,False
8,9,OpenDNS,208.67.222.222,208.67.220.220,False
9,10,OpenNIC,185.121.177.177,169.239.202.202,False


In [16]:
dot_err_filtered = dot_err[~dot_err['resolver'].isin(['91.239.100.100', '89.233.43.71'])]

In [17]:
dot_err_filtered

,msm_id,resolver,prb_id,src_addr,dst_addr,is_tls,return_code,err,err_msg
16522983,22217033,None,10007,None,None,1,None,1,{u'timeout': 5000}
16522984,22217033,None,10007,None,None,1,None,1,{u'timeout': 5000}
16522985,22217033,None,10007,None,None,1,None,1,{u'timeout': 5000}
16522986,22217033,None,10019,None,None,1,None,1,{u'TU_BAD_ADDR': True}
16522987,22217033,None,10023,None,None,1,None,1,{u'TU_BAD_ADDR': True}
...,...,...,...,...,...,...,...,...,...
160697073,22299011,9.9.9.9,35600,None,None,1,None,1,{u'timeout': 5000}
160697080,22299011,9.9.9.9,35626,None,None,1,None,1,{u'timeout': 5000}
160697093,22299011,9.9.9.9,35674,None,None,1,None,1,{u'timeout': 5000}
160697104,22299011,9.9.9.9,35709,None,None,1,None,1,{u'timeout': 5000}


In [18]:
dot_err_filtered.groupby(['prb_id']).size().sort_values(ascending=False)

prb_id
26718    9782
21569    9782
33510    9782
34327    9782
33643    9782
         ... 
18213       1
15505       1
34458       1
25322       1
34096       1
Length: 4924, dtype: int64

In [19]:
dot_tuconnect_filtered = dot_err_filtered[dot_err_filtered['err_msg'].str.contains('TUCONNECT')]

In [20]:
dot_tuconnect_filtered.groupby(['prb_id']).size().sort_values(ascending=False)

prb_id
26451    9782
33408    8386
22955    8386
31088    8386
20461    8381
         ... 
13593       1
23279       1
23290       1
13646       1
12908       1
Length: 1832, dtype: int64

In [21]:
dot_tuconnect_filtered

,msm_id,resolver,prb_id,src_addr,dst_addr,is_tls,return_code,err,err_msg
16522990,22217033,None,10031,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}
16522998,22217033,None,10051,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}
16522999,22217033,None,10051,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}
16523000,22217033,None,10051,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}
16523001,22217033,None,10052,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}
...,...,...,...,...,...,...,...,...,...
160696705,22299011,9.9.9.9,33408,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}
160696735,22299011,9.9.9.9,33510,None,None,1,None,1,{u'TUCONNECT': u'No route to host'}
160696768,22299011,9.9.9.9,33643,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}
160696988,22299011,9.9.9.9,34871,None,None,1,None,1,{u'TUCONNECT': u'Connection refused'}


In [22]:
tls_err = pd.crosstab(dot_tuconnect_filtered['prb_id'], dot_tuconnect_filtered['resolver'], margins=True)

In [23]:
tls_err

resolver,1.1.1.1,185.228.168.168,8.8.8.8,9.9.9.9,All
prb_id,,,,,
10019,0,23,0,0,23
10048,1398,1400,1398,1398,5594
10058,1398,1400,1398,1398,5594
10180,1398,0,0,0,1398
10203,0,50,0,0,50
...,...,...,...,...,...
35723,1,0,0,0,1
35724,1,0,0,0,1
35726,0,1,0,0,1


In [24]:
tls_err[tls_err['All'] > 1000]

resolver,1.1.1.1,185.228.168.168,8.8.8.8,9.9.9.9,All
prb_id,,,,,
10048,1398,1400,1398,1398,5594
10058,1398,1400,1398,1398,5594
10180,1398,0,0,0,1398
10215,1396,1,0,0,1397
10243,1398,1,0,0,1399
...,...,...,...,...,...
35151,1398,0,0,0,1398
35577,1315,0,0,0,1315
35685,1386,0,0,0,1386


In [25]:
tls_err[tls_err['9.9.9.9'] > 1000]

resolver,1.1.1.1,185.228.168.168,8.8.8.8,9.9.9.9,All
prb_id,,,,,
10048,1398,1400,1398,1398,5594
10058,1398,1400,1398,1398,5594
11608,1199,1200,1198,1198,4795
11743,1398,1400,1398,1398,5594
12087,1397,1400,1398,1398,5593
12321,1398,1400,1398,1398,5594
12880,1126,1123,1116,1133,4498
15355,1398,0,1398,1398,4194
15762,1394,1396,1396,1396,5582


In [26]:
len(tls_err[tls_err['9.9.9.9'] > 1000])

34

^ this includes "All", so 33 probes in total, save to `../metadata/faulty-probes.csv`

In [27]:
dot_err[dot_err['resolver'] == '185.228.168.168']['err_msg'].value_counts()  # example for CleanBrowsing

{u'timeout': 5000}                             718412
{u'TUCONNECT': u'Connection refused'}           41475
{u'TUCONNECT': u'Connection reset by peer'}      3153
{u'TUCONNECT': u'Success'}                       3035
{u'TUCONNECT': u'No route to host'}              2014
{u'TUCONNECT': u'Network is unreachable'}         323
Name: err_msg, dtype: int64